In [1]:
import numpy as np
from tensorflow import keras

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler, PowerTransformer
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

<h2>Обработка данных</h2>

In [2]:
df = pd.read_csv('../data/df_split_by_map_part_info.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1813 entries, 0 to 1812
Data columns (total 52 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   map                             1813 non-null   object 
 1   result                          1813 non-null   int64  
 2   team1_world_rank                1813 non-null   int64  
 3   team2_world_rank                1813 non-null   int64  
 4   1_top_Times played              1813 non-null   int64  
 5   1_top_wins                      1813 non-null   int64  
 6   1_top_draws                     1813 non-null   int64  
 7   1_top_losses                    1813 non-null   int64  
 8   1_top_Total rounds played       1813 non-null   int64  
 9   1_top_Rounds won                1813 non-null   int64  
 10  1_top_Win percent               1813 non-null   float64
 11  1_top_Pistol rounds             1813 non-null   int64  
 12  1_top_Pistol rounds won         18

In [3]:
df.head()

,map,result,team1_world_rank,team2_world_rank,1_top_Times played,1_top_wins,1_top_draws,1_top_losses,1_top_Total rounds played,1_top_Rounds won,...,2_All_draws,2_All_losses,2_All_Total rounds played,2_All_Rounds won,2_All_Win percent,2_All_Pistol rounds,2_All_Pistol rounds won,2_All_Pistol round win percent,2_All_CT round win percent,2_All_T round win percent
0,Nuke,-1,28,46,3,0,0,3,80,32,...,0,2,140,82,0.667,12,7,0.583,0.774,0.436
1,Ancient,-1,28,46,1,0,0,1,27,11,...,0,2,135,86,0.667,12,8,0.667,0.756,0.578
2,Mirage,1,1,223,7,4,0,3,182,106,...,0,2,151,80,0.667,12,7,0.583,0.556,0.511
3,Ancient,1,1,223,2,2,0,0,54,32,...,0,0,0,0,0.000,0,0,0.000,0.000,0.000
4,Overpass,-1,14,101,4,2,0,2,106,60,...,0,0,0,0,0.000,0,0,0.000,0.000,0.000


In [4]:
maps_decoder = LabelEncoder()
df['map'] = maps_decoder.fit_transform(df['map'])

In [5]:
cv = KFold(shuffle=True, random_state=1)
X = df.drop('result', axis=1)
y = pd.get_dummies(df['result'])

In [6]:
X_scaled = pd.DataFrame()
transform = StandardScaler()
for i in X.columns:
    X_scaled[i] = transform.fit_transform(np.array(X[i]).reshape(-1, 1)).reshape(1, -1)[0]
X_scaled.head()

,map,team1_world_rank,team2_world_rank,1_top_Times played,1_top_wins,1_top_draws,1_top_losses,1_top_Total rounds played,1_top_Rounds won,1_top_Win percent,...,2_All_draws,2_All_losses,2_All_Total rounds played,2_All_Rounds won,2_All_Win percent,2_All_Pistol rounds,2_All_Pistol rounds won,2_All_Pistol round win percent,2_All_CT round win percent,2_All_T round win percent
0,0.512696,-0.640977,-0.542983,-0.542942,-0.962287,0.0,0.172220,-0.543115,-0.656152,-1.388941,...,0.0,-0.381858,-0.273208,-0.128951,0.641503,-0.119377,0.043827,0.543694,1.386714,0.149444
1,-1.587910,-0.640977,-0.542983,-0.944309,-0.962287,0.0,-0.649451,-0.936015,-0.952151,-1.388941,...,0.0,-0.381858,-0.309503,-0.074231,0.641503,-0.119377,0.224002,0.913605,1.294443,0.985356
2,-0.012455,-1.093133,2.217271,0.259793,0.269576,0.0,0.172220,0.213033,0.386890,0.431672,...,0.0,-0.381858,-0.193359,-0.156310,0.641503,-0.119377,0.043827,0.543694,0.269208,0.590947
3,-1.587910,-1.093133,2.217271,-0.743625,-0.346355,0.0,-1.060287,-0.735858,-0.656152,1.799523,...,0.0,-1.180369,-1.289469,-1.250692,-1.582216,-1.299904,-1.217403,-2.023658,-2.580946,-2.417161
4,1.037848,-0.875428,0.314723,-0.342258,-0.346355,0.0,-0.238616,-0.350371,-0.261488,0.205291,...,0.0,-1.180369,-1.289469,-1.250692,-1.582216,-1.299904,-1.217403,-2.023658,-2.580946,-2.417161


In [7]:
X_norm = pd.DataFrame()
transform = PowerTransformer()
for i in X.columns:
    X_norm[i] = transform.fit_transform(np.array(X[i]).reshape(-1, 1)).reshape(1, -1)[0]
X_norm.head()

,map,team1_world_rank,team2_world_rank,1_top_Times played,1_top_wins,1_top_draws,1_top_losses,1_top_Total rounds played,1_top_Rounds won,1_top_Win percent,...,2_All_draws,2_All_losses,2_All_Total rounds played,2_All_Rounds won,2_All_Win percent,2_All_Pistol rounds,2_All_Pistol rounds won,2_All_Pistol round win percent,2_All_CT round win percent,2_All_T round win percent
0,0.546561,-0.461564,-0.325356,-0.329416,-1.372699,0.0,0.461334,-0.227559,-0.390533,-1.401007,...,0.0,-0.168699,-0.015431,0.131736,0.628533,0.106223,0.289221,0.509074,2.046168,0.021071
1,-1.697826,-0.461564,-0.325356,-1.014705,-1.372699,0.0,-0.511795,-0.857022,-0.954480,-1.401007,...,0.0,-0.168699,-0.051636,0.182263,0.628533,0.106223,0.448382,0.927165,1.854781,1.103045
2,0.048231,-2.183894,1.700456,0.505794,0.617370,0.0,0.461334,0.478438,0.616456,0.441944,...,0.0,-0.168699,0.061945,0.105980,0.628533,0.106223,0.289221,0.509074,0.121955,0.562669
3,-1.697826,-2.183894,1.700456,-0.630374,-0.017951,0.0,-1.416822,-0.489042,-0.390533,1.769659,...,0.0,-1.556556,-1.922741,-1.871938,-1.544899,-1.789572,-1.678101,-1.858974,-2.047743,-2.024563
4,1.022167,-0.992090,0.552079,-0.078349,-0.017951,0.0,0.048862,-0.012298,0.077822,0.218090,...,0.0,-1.556556,-1.922741,-1.871938,-1.544899,-1.789572,-1.678101,-1.858974,-2.047743,-2.024563


In [8]:
transform = PCA(n_components=50)
X_pca = pd.DataFrame(transform.fit_transform(X))
transform.explained_variance_ratio_.sum()

1.0

In [9]:
X_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-154.046850,-65.920865,117.943493,8.399808,-37.880471,-1.076925,12.121085,-6.135297,-4.494921,0.271992,...,1.679759e-14,1.496955e-13,6.119278e-14,-9.535896e-14,-1.880368e-13,2.000170e-14,6.125394e-14,-8.955362e-14,-2.339807e-14,-2.927347e-14
1,-231.528707,-49.613801,130.897567,12.530517,-39.882945,3.813638,3.424059,-10.627984,-7.665576,0.599226,...,1.847680e-14,2.937373e-13,1.419622e-13,-3.045945e-13,-1.291297e-13,2.305459e-13,8.363738e-14,1.622983e-13,-5.812063e-14,1.138158e-14
2,12.903208,-18.776773,-76.809442,-49.188327,-8.168537,143.039846,-13.789771,-5.193121,0.800527,-0.235483,...,1.964673e-14,-4.929491e-14,1.449009e-13,1.117522e-13,1.423150e-13,9.269562e-14,1.923379e-14,6.234012e-15,-1.786345e-14,2.135707e-14
3,-234.506347,-202.420329,-0.232122,-4.073161,-6.196977,138.501382,-4.066865,-1.766936,0.253601,0.073289,...,-4.494166e-13,-6.209447e-14,-7.107057e-14,-1.517081e-14,-1.146200e-13,-1.630623e-14,-9.442085e-14,-8.011682e-14,8.098731e-14,-3.300123e-15
4,-146.412043,-202.565628,18.492581,49.599850,-50.167326,42.875968,-5.035037,-1.441627,-0.072126,0.849719,...,5.023642e-14,3.192910e-13,8.777631e-14,-1.677740e-13,-8.895281e-14,1.190552e-13,1.127519e-13,8.961179e-14,-1.120629e-14,-7.770314e-15


<h2>Обучение моделей</h2>

<h3>Данные без обработки</h3>

<b>Данные без обработки - ручками подбираем параметры, при которых нет переобучения.</b>

In [16]:
%%time
acc = []
units = 5
layers = 3
dropout = 1 #умножается на 0.1
epoch = 50

X_splitter = X
y_splitter = pd.get_dummies(y)

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_splitter, y_splitter, test_size=0.2, random_state=i)
    model = Sequential()
    model.add(Dense(input_dim=X_train.shape[1], units=units,
                     kernel_initializer='normal', bias_initializer='zeros'))
    model.add(Activation('relu'))

    for i in range(0, layers):
        model.add(Dense(units=units, kernel_initializer='normal',
                         bias_initializer='zeros'))
        model.add(Activation('relu'))
        model.add(Dropout(dropout*0.1))

    model.add(Dense(units=2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epoch, verbose=1)
    acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
    print(f"accuracy: {acc[-1]}")
print(f'units: {units}, layers: {layers}, dropout: {dropout}, mean acc: {np.mean(acc)}')

Epoch 1/50
46/46 [==============================] - 0s 779us/step - loss: 0.6915 - accuracy: 0.5455
Epoch 2/50
46/46 [==============================] - 0s 734us/step - loss: 0.6855 - accuracy: 0.5469
Epoch 3/50
46/46 [==============================] - 0s 734us/step - loss: 0.6783 - accuracy: 0.5469
Epoch 4/50
46/46 [==============================] - 0s 769us/step - loss: 0.6794 - accuracy: 0.5469
Epoch 5/50
46/46 [==============================] - 0s 756us/step - loss: 0.6777 - accuracy: 0.5469
Epoch 6/50
46/46 [==============================] - 0s 756us/step - loss: 0.6786 - accuracy: 0.5469
Epoch 7/50
46/46 [==============================] - 0s 769us/step - loss: 0.6742 - accuracy: 0.5469
Epoch 8/50
46/46 [==============================] - 0s 756us/step - loss: 0.6750 - accuracy: 0.5469
Epoch 9/50
46/46 [==============================] - 0s 734us/step - loss: 0.6787 - accuracy: 0.5469
Epoch 10/50
46/46 [==============================] - 0s 756us/step - loss: 0.6771 - accuracy: 0.5469

46/46 [==============================] - 0s 734us/step - loss: 0.6665 - accuracy: 0.5986
Epoch 32/50
46/46 [==============================] - 0s 756us/step - loss: 0.6679 - accuracy: 0.5897
Epoch 33/50
46/46 [==============================] - 0s 756us/step - loss: 0.6706 - accuracy: 0.5883
Epoch 34/50
46/46 [==============================] - 0s 756us/step - loss: 0.6642 - accuracy: 0.6048
Epoch 35/50
46/46 [==============================] - 0s 756us/step - loss: 0.6694 - accuracy: 0.5910
Epoch 36/50
46/46 [==============================] - 0s 734us/step - loss: 0.6682 - accuracy: 0.6021
Epoch 37/50
46/46 [==============================] - 0s 734us/step - loss: 0.6654 - accuracy: 0.5931
Epoch 38/50
46/46 [==============================] - 0s 756us/step - loss: 0.6654 - accuracy: 0.5917
Epoch 39/50
46/46 [==============================] - 0s 756us/step - loss: 0.6663 - accuracy: 0.6000
Epoch 40/50
46/46 [==============================] - 0s 734us/step - loss: 0.6630 - accuracy: 0.5986
Ep

46/46 [==============================] - 0s 778us/step - loss: 0.6663 - accuracy: 0.5966
Epoch 11/50
46/46 [==============================] - 0s 778us/step - loss: 0.6693 - accuracy: 0.5938
Epoch 12/50
46/46 [==============================] - 0s 778us/step - loss: 0.6657 - accuracy: 0.6069
Epoch 13/50
46/46 [==============================] - 0s 778us/step - loss: 0.6647 - accuracy: 0.6000
Epoch 14/50
46/46 [==============================] - 0s 801us/step - loss: 0.6635 - accuracy: 0.5924
Epoch 15/50
46/46 [==============================] - 0s 756us/step - loss: 0.6631 - accuracy: 0.6007
Epoch 16/50
46/46 [==============================] - 0s 756us/step - loss: 0.6644 - accuracy: 0.6021
Epoch 17/50
46/46 [==============================] - 0s 801us/step - loss: 0.6644 - accuracy: 0.6028
Epoch 18/50
46/46 [==============================] - 0s 734us/step - loss: 0.6647 - accuracy: 0.6034
Epoch 19/50
46/46 [==============================] - 0s 756us/step - loss: 0.6672 - accuracy: 0.6117
Ep

46/46 [==============================] - 0s 756us/step - loss: 0.6695 - accuracy: 0.5993
Epoch 41/50
46/46 [==============================] - 0s 756us/step - loss: 0.6669 - accuracy: 0.5917
Epoch 42/50
46/46 [==============================] - 0s 778us/step - loss: 0.6687 - accuracy: 0.5972
Epoch 43/50
46/46 [==============================] - 0s 756us/step - loss: 0.6686 - accuracy: 0.5972
Epoch 44/50
46/46 [==============================] - 0s 756us/step - loss: 0.6660 - accuracy: 0.5897
Epoch 45/50
46/46 [==============================] - 0s 778us/step - loss: 0.6653 - accuracy: 0.5952
Epoch 46/50
46/46 [==============================] - 0s 768us/step - loss: 0.6655 - accuracy: 0.6055
Epoch 47/50
46/46 [==============================] - 0s 778us/step - loss: 0.6663 - accuracy: 0.6090
Epoch 48/50
46/46 [==============================] - 0s 779us/step - loss: 0.6657 - accuracy: 0.6028
Epoch 49/50
46/46 [==============================] - 0s 769us/step - loss: 0.6670 - accuracy: 0.6103
Ep

Вроде бы нет переобучения, так что можем начать подбирать параметры в ближайшем промежутке

In [18]:
results = pd.DataFrame({'units': [], 'layers': [], 'dropout': [], 'accuracy_mean': [], 'accuracy_std': []})

In [19]:
%%time
X_splitter = X
y_splitter = pd.get_dummies(y)

for units in range(5,10):
    for layers in range(2,6):
        for dropout in range(1,5):
            acc = []
            for i in range(4):
                X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=i)
                model = Sequential()
                model.add(Dense(input_dim=X_train.shape[1], units=units,
                                 kernel_initializer='normal', bias_initializer='zeros'))
                model.add(Activation('relu'))

                for i in range(0, layers):
                    model.add(Dense(units=units, kernel_initializer='normal',
                                     bias_initializer='zeros'))
                    model.add(Activation('relu'))
                    model.add(Dropout(dropout*0.1))

                model.add(Dense(units=2))
                model.add(Activation('softmax'))

                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                model.fit(X_train, y_train, epochs=50, verbose=0)
                acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
                print(f'units: {units}, layers: {layers}, dropout: {dropout}, accuracy: {acc[-1]}')
                
            results = results.append({'units': units, 
                                      'layers': layers, 
                                      'dropout': dropout, 
                                      'accuracy_mean': np.mean(acc), 
                                      'accuracy_std': np.std(acc)}, 
                                      ignore_index=True)
            print(f'units: {units}, layers: {layers}, dropout: {dropout}, ----------------------------mean acc: {np.mean(acc)}')

15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6035242290748899
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6277533039647577
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.5925110132158591
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6079295154185022
units: 5, layers: 2, dropout: 1, ----------------------------mean acc: 0.6079295154185022
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.5682819383259912
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.6211453744493393
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.6189427312775331
15/15 [==============================] - 0s 572us/step
units: 5, layers: 2, dropout: 2, acc

15/15 [==============================] - 0s 643us/step
units: 5, layers: 5, dropout: 3, accuracy: 0.5110132158590308
units: 5, layers: 5, dropout: 3, ----------------------------mean acc: 0.551762114537445
15/15 [==============================] - 0s 643us/step
units: 5, layers: 5, dropout: 4, accuracy: 0.5594713656387665
15/15 [==============================] - 0s 572us/step
units: 5, layers: 5, dropout: 4, accuracy: 0.5770925110132159
15/15 [==============================] - 0s 643us/step
units: 5, layers: 5, dropout: 4, accuracy: 0.5594713656387665
15/15 [==============================] - 0s 572us/step
units: 5, layers: 5, dropout: 4, accuracy: 0.5110132158590308
units: 5, layers: 5, dropout: 4, ----------------------------mean acc: 0.551762114537445
15/15 [==============================] - 0s 500us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.5616740088105727
15/15 [==============================] - 0s 572us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.6101321585903083
15/

15/15 [==============================] - 0s 572us/step
units: 6, layers: 5, dropout: 2, accuracy: 0.5881057268722467
15/15 [==============================] - 0s 643us/step
units: 6, layers: 5, dropout: 2, accuracy: 0.5110132158590308
units: 6, layers: 5, dropout: 2, ----------------------------mean acc: 0.5655286343612336
15/15 [==============================] - 0s 643us/step
units: 6, layers: 5, dropout: 3, accuracy: 0.5594713656387665
15/15 [==============================] - 0s 643us/step
units: 6, layers: 5, dropout: 3, accuracy: 0.6343612334801763
15/15 [==============================] - 0s 643us/step
units: 6, layers: 5, dropout: 3, accuracy: 0.5594713656387665
15/15 [==============================] - 0s 572us/step
units: 6, layers: 5, dropout: 3, accuracy: 0.5110132158590308
units: 6, layers: 5, dropout: 3, ----------------------------mean acc: 0.5660792951541851
15/15 [==============================] - 0s 572us/step
units: 6, layers: 5, dropout: 4, accuracy: 0.566079295154185
15

15/15 [==============================] - 0s 643us/step
units: 7, layers: 5, dropout: 1, accuracy: 0.5991189427312775
15/15 [==============================] - 0s 572us/step
units: 7, layers: 5, dropout: 1, accuracy: 0.5947136563876652
15/15 [==============================] - 0s 572us/step
units: 7, layers: 5, dropout: 1, accuracy: 0.5991189427312775
units: 7, layers: 5, dropout: 1, ----------------------------mean acc: 0.5930616740088106
15/15 [==============================] - 0s 715us/step
units: 7, layers: 5, dropout: 2, accuracy: 0.5748898678414097
15/15 [==============================] - 0s 643us/step
units: 7, layers: 5, dropout: 2, accuracy: 0.5770925110132159
15/15 [==============================] - 0s 572us/step
units: 7, layers: 5, dropout: 2, accuracy: 0.5594713656387665
15/15 [==============================] - 0s 644us/step
units: 7, layers: 5, dropout: 2, accuracy: 0.5110132158590308
units: 7, layers: 5, dropout: 2, ----------------------------mean acc: 0.5556167400881058
1

15/15 [==============================] - 0s 572us/step
units: 8, layers: 4, dropout: 4, accuracy: 0.6013215859030837
15/15 [==============================] - 0s 572us/step
units: 8, layers: 4, dropout: 4, accuracy: 0.6079295154185022
15/15 [==============================] - 0s 572us/step
units: 8, layers: 4, dropout: 4, accuracy: 0.5903083700440529
15/15 [==============================] - 0s 572us/step
units: 8, layers: 4, dropout: 4, accuracy: 0.5110132158590308
units: 8, layers: 4, dropout: 4, ----------------------------mean acc: 0.5776431718061674
15/15 [==============================] - 0s 643us/step
units: 8, layers: 5, dropout: 1, accuracy: 0.5859030837004405
15/15 [==============================] - 0s 644us/step
units: 8, layers: 5, dropout: 1, accuracy: 0.6167400881057269
15/15 [==============================] - 0s 643us/step
units: 8, layers: 5, dropout: 1, accuracy: 0.6035242290748899
15/15 [==============================] - 0s 572us/step
units: 8, layers: 5, dropout: 1, acc

15/15 [==============================] - 0s 572us/step
units: 9, layers: 4, dropout: 2, accuracy: 0.5969162995594713
units: 9, layers: 4, dropout: 2, ----------------------------mean acc: 0.5936123348017621
15/15 [==============================] - 0s 643us/step
units: 9, layers: 4, dropout: 3, accuracy: 0.5859030837004405
15/15 [==============================] - 0s 572us/step
units: 9, layers: 4, dropout: 3, accuracy: 0.6277533039647577
15/15 [==============================] - 0s 643us/step
units: 9, layers: 4, dropout: 3, accuracy: 0.5770925110132159
15/15 [==============================] - 0s 643us/step
units: 9, layers: 4, dropout: 3, accuracy: 0.5110132158590308
units: 9, layers: 4, dropout: 3, ----------------------------mean acc: 0.5754405286343612
15/15 [==============================] - 0s 643us/step
units: 9, layers: 4, dropout: 4, accuracy: 0.5881057268722467
15/15 [==============================] - 0s 643us/step
units: 9, layers: 4, dropout: 4, accuracy: 0.6233480176211453
1

In [20]:
results.sort_values('accuracy_mean', ascending=False).head(20)

,units,layers,dropout,accuracy_mean,accuracy_std
52,8.0,3.0,1.0,0.614537,0.012749
64,9.0,2.0,1.0,0.609031,0.008459
60,8.0,5.0,1.0,0.607930,0.014939
0,5.0,2.0,1.0,0.607930,0.012749
70,9.0,3.0,3.0,0.607930,0.012164
4,5.0,3.0,1.0,0.606828,0.012313
21,6.0,3.0,2.0,0.606278,0.012301
20,6.0,3.0,1.0,0.604075,0.020626
49,8.0,2.0,2.0,0.604075,0.013876
57,8.0,4.0,2.0,0.604075,0.021433


Нашли наилучшие параметры, правда у них у всех большая дисперсия, что значит, что на других выборках они могут давать плохие результаты. Но больше мы на такое большое время запускать поиск не будем.

<h2>Данные со стандартизацией</h2>

In [27]:
%%time
acc = []
units = 5
layers = 3
dropout = 1 #умножается на 0.1
epoch = 50

X_splitter = X_scaled.copy()
y_splitter = pd.get_dummies(y)

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_splitter, y_splitter, test_size=0.2, random_state=i)
    model = Sequential()
    model.add(Dense(input_dim=X_train.shape[1], units=units,
                     kernel_initializer='normal', bias_initializer='zeros'))
    model.add(Activation('relu'))

    for i in range(0, layers):
        model.add(Dense(units=units, kernel_initializer='normal',
                         bias_initializer='zeros'))
        model.add(Activation('relu'))
        model.add(Dropout(dropout*0.1))

    model.add(Dense(units=2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epoch, verbose=1)
    acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
    print(f"accuracy: {acc[-1]}")
print(f'units: {units}, layers: {layers}, dropout: {dropout}, mean acc: {np.mean(acc)}')

Epoch 1/50
46/46 [==============================] - 0s 801us/step - loss: 0.6920 - accuracy: 0.5483
Epoch 2/50
46/46 [==============================] - 0s 778us/step - loss: 0.6903 - accuracy: 0.5469
Epoch 3/50
46/46 [==============================] - 0s 801us/step - loss: 0.6889 - accuracy: 0.5469
Epoch 4/50
46/46 [==============================] - 0s 734us/step - loss: 0.6848 - accuracy: 0.5469
Epoch 5/50
46/46 [==============================] - 0s 823us/step - loss: 0.6775 - accuracy: 0.5469
Epoch 6/50
46/46 [==============================] - 0s 801us/step - loss: 0.6745 - accuracy: 0.5469
Epoch 7/50
46/46 [==============================] - 0s 801us/step - loss: 0.6736 - accuracy: 0.5469
Epoch 8/50
46/46 [==============================] - 0s 801us/step - loss: 0.6720 - accuracy: 0.5469
Epoch 9/50
46/46 [==============================] - 0s 801us/step - loss: 0.6728 - accuracy: 0.5469
Epoch 10/50
46/46 [==============================] - 0s 801us/step - loss: 0.6705 - accuracy: 0.5834

46/46 [==============================] - 0s 801us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 32/50
46/46 [==============================] - 0s 801us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 33/50
46/46 [==============================] - 0s 791us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 34/50
46/46 [==============================] - 0s 778us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 35/50
46/46 [==============================] - 0s 778us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 36/50
46/46 [==============================] - 0s 756us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 37/50
46/46 [==============================] - 0s 734us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 38/50
46/46 [==============================] - 0s 778us/step - loss: 0.6890 - accuracy: 0.5462
Epoch 39/50
46/46 [==============================] - 0s 778us/step - loss: 0.6889 - accuracy: 0.5462
Epoch 40/50
46/46 [==============================] - 0s 756us/step - loss: 0.6889 - accuracy: 0.5462
Ep

46/46 [==============================] - 0s 778us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 11/50
46/46 [==============================] - 0s 768us/step - loss: 0.6859 - accuracy: 0.5607
Epoch 12/50
46/46 [==============================] - 0s 823us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 13/50
46/46 [==============================] - 0s 779us/step - loss: 0.6859 - accuracy: 0.5607
Epoch 14/50
46/46 [==============================] - 0s 791us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 15/50
46/46 [==============================] - 0s 756us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 16/50
46/46 [==============================] - 0s 801us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 17/50
46/46 [==============================] - 0s 778us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 18/50
46/46 [==============================] - 0s 823us/step - loss: 0.6858 - accuracy: 0.5607
Epoch 19/50
46/46 [==============================] - 0s 801us/step - loss: 0.6858 - accuracy: 0.5607
Ep

46/46 [==============================] - 0s 801us/step - loss: 0.6537 - accuracy: 0.6359
Epoch 41/50
46/46 [==============================] - 0s 778us/step - loss: 0.6552 - accuracy: 0.6345
Epoch 42/50
46/46 [==============================] - 0s 778us/step - loss: 0.6555 - accuracy: 0.6317
Epoch 43/50
46/46 [==============================] - 0s 756us/step - loss: 0.6573 - accuracy: 0.6200
Epoch 44/50
46/46 [==============================] - 0s 756us/step - loss: 0.6520 - accuracy: 0.6269
Epoch 45/50
46/46 [==============================] - 0s 778us/step - loss: 0.6475 - accuracy: 0.6207
Epoch 46/50
46/46 [==============================] - 0s 756us/step - loss: 0.6547 - accuracy: 0.6234
Epoch 47/50
46/46 [==============================] - 0s 756us/step - loss: 0.6536 - accuracy: 0.6248
Epoch 48/50
46/46 [==============================] - 0s 756us/step - loss: 0.6492 - accuracy: 0.6248
Epoch 49/50
46/46 [==============================] - 0s 778us/step - loss: 0.6497 - accuracy: 0.6310
Ep

In [28]:
%%time
X_splitter = X_scaled.copy()
y_splitter = pd.get_dummies(y)
results = pd.DataFrame({'units': [], 'layers': [], 'dropout': [], 'accuracy_mean': [], 'accuracy_std': []})
for units in range(5,15):
    for layers in range(2,6):
        for dropout in range(1,4):
            acc = []
            for i in range(5):
                X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=i)
                model = Sequential()
                model.add(Dense(input_dim=X_train.shape[1], units=units,
                                 kernel_initializer='normal', bias_initializer='zeros'))
                model.add(Activation('relu'))

                for i in range(0, layers):
                    model.add(Dense(units=units, kernel_initializer='normal',
                                     bias_initializer='zeros'))
                    model.add(Activation('relu'))
                    model.add(Dropout(dropout*0.1))

                model.add(Dense(units=2))
                model.add(Activation('softmax'))

                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                model.fit(X_train, y_train, epochs=50, verbose=0)
                acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
                print(f'units: {units}, layers: {layers}, dropout: {dropout}, accuracy: {acc[-1]}')
                
            results = results.append({'units': units, 
                                      'layers': layers, 
                                      'dropout': dropout, 
                                      'accuracy_mean': np.mean(acc), 
                                      'accuracy_std': np.std(acc)}, 
                                      ignore_index=True)
            print(f'units: {units}, layers: {layers}, dropout: {dropout}, mean acc: {np.mean(acc)}')

12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 546us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.5482093663911846
12/12 [==============================] - 0s 546us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6198347107438017
units: 5, layers: 2, dropout: 1, mean acc: 0.584022038567493
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.5647382920110193
12/12 [==============================] - 0s 546us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.6170798898071626
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.581267217630854
12/1

12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.5785123966942148
units: 6, layers: 2, dropout: 1, mean acc: 0.5878787878787878
12/12 [==============================] - 0s 637us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.581267217630854
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.6391184573002755
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.5674931129476584
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.6115702479338843
units: 6, layers: 2, dropout: 2, mean acc: 0.5972451790633609
12/12 [==============================] - 0s 637us/step
units: 6, layers: 2, dropout: 3, accuracy: 0.5757575757575758
12/12 [==============================] - 0s 637us/step
uni

12/12 [==============================] - 0s 546us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 546us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 728us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.6170798898071626
units: 7, layers: 2, dropout: 2, mean acc: 0.5961432506887052
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 3, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 3, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 3, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 3, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 3, accuracy: 0.6143250688705234
un

12/12 [==============================] - 0s 546us/step
units: 8, layers: 2, dropout: 3, accuracy: 0.5537190082644629
12/12 [==============================] - 0s 546us/step
units: 8, layers: 2, dropout: 3, accuracy: 0.6115702479338843
12/12 [==============================] - 0s 546us/step
units: 8, layers: 2, dropout: 3, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 3, accuracy: 0.5619834710743802
12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 3, accuracy: 0.5950413223140496
units: 8, layers: 2, dropout: 3, mean acc: 0.5851239669421486
12/12 [==============================] - 0s 637us/step
units: 8, layers: 3, dropout: 1, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 686us/step
units: 8, layers: 3, dropout: 1, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 637us/step
units: 8, layers: 3, dropout: 1, accuracy: 0.5950413223140496
12

12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 3, accuracy: 0.6198347107438017
units: 9, layers: 2, dropout: 3, mean acc: 0.5994490358126722
12/12 [==============================] - 0s 637us/step
units: 9, layers: 3, dropout: 1, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 9, layers: 3, dropout: 1, accuracy: 0.6088154269972452
12/12 [==============================] - 0s 637us/step
units: 9, layers: 3, dropout: 1, accuracy: 0.6060606060606061
12/12 [==============================] - 0s 546us/step
units: 9, layers: 3, dropout: 1, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 9, layers: 3, dropout: 1, accuracy: 0.6088154269972452
units: 9, layers: 3, dropout: 1, mean acc: 0.5994490358126722
12/12 [==============================] - 0s 637us/step
units: 9, layers: 3, dropout: 2, accuracy: 0.5730027548209367
12/12 [==============================] - 0s 546us/step
un

12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 1, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 1, accuracy: 0.5895316804407713
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 1, accuracy: 0.6033057851239669
units: 10, layers: 3, dropout: 1, mean acc: 0.596694214876033
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 2, accuracy: 0.5647382920110193
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 2, accuracy: 0.6088154269972452
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 2, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 2, accuracy: 0.6005509641873278
12/12 [==============================] - 0s 637us/step
units: 10, layers: 3, dropout: 2, accuracy: 0.61707988980

12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 2, accuracy: 0.6005509641873278
12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 2, accuracy: 0.6253443526170799
12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 2, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 2, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 2, accuracy: 0.5867768595041323
units: 11, layers: 3, dropout: 2, mean acc: 0.5950413223140496
12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 3, accuracy: 0.6060606060606061
12/12 [==============================] - 0s 637us/step
units: 11, layers: 3, dropout: 3, accuracy: 0.6473829201101928
12/12 [==============================] - 0s 546us/step
units: 11, layers: 3, dropout: 3, accuracy: 0.5867768595

12/12 [==============================] - 0s 637us/step
units: 12, layers: 3, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 546us/step
units: 12, layers: 3, dropout: 2, accuracy: 0.6115702479338843
units: 12, layers: 3, dropout: 2, mean acc: 0.5928374655647384
12/12 [==============================] - 0s 637us/step
units: 12, layers: 3, dropout: 3, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 12, layers: 3, dropout: 3, accuracy: 0.6005509641873278
12/12 [==============================] - 0s 637us/step
units: 12, layers: 3, dropout: 3, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 12, layers: 3, dropout: 3, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 637us/step
units: 12, layers: 3, dropout: 3, accuracy: 0.5895316804407713
units: 12, layers: 3, dropout: 3, mean acc: 0.5834710743801652
12/12 [==============================] - 0s 637u

12/12 [==============================] - 0s 728us/step
units: 13, layers: 3, dropout: 3, accuracy: 0.6115702479338843
12/12 [==============================] - 0s 637us/step
units: 13, layers: 3, dropout: 3, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 637us/step
units: 13, layers: 3, dropout: 3, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 546us/step
units: 13, layers: 3, dropout: 3, accuracy: 0.5950413223140496
units: 13, layers: 3, dropout: 3, mean acc: 0.5933884297520662
12/12 [==============================] - 0s 637us/step
units: 13, layers: 4, dropout: 1, accuracy: 0.5619834710743802
12/12 [==============================] - 0s 637us/step
units: 13, layers: 4, dropout: 1, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 13, layers: 4, dropout: 1, accuracy: 0.6060606060606061
12/12 [==============================] - 0s 637us/step
units: 13, layers: 4, dropout: 1, accuracy: 0.5674931129

12/12 [==============================] - 0s 546us/step
units: 14, layers: 3, dropout: 3, accuracy: 0.5867768595041323
units: 14, layers: 3, dropout: 3, mean acc: 0.5928374655647384
12/12 [==============================] - 0s 637us/step
units: 14, layers: 4, dropout: 1, accuracy: 0.559228650137741
12/12 [==============================] - 0s 637us/step
units: 14, layers: 4, dropout: 1, accuracy: 0.6225895316804407
12/12 [==============================] - 0s 728us/step
units: 14, layers: 4, dropout: 1, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 14, layers: 4, dropout: 1, accuracy: 0.5537190082644629
12/12 [==============================] - 0s 728us/step
units: 14, layers: 4, dropout: 1, accuracy: 0.6170798898071626
units: 14, layers: 4, dropout: 1, mean acc: 0.5911845730027548
12/12 [==============================] - 0s 640us/step
units: 14, layers: 4, dropout: 2, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 728us

In [29]:
results.sort_values('accuracy_mean', ascending=False).head(20)

,units,layers,dropout,accuracy_mean,accuracy_std
60,10.0,2.0,1.0,0.608815,0.012684
119,14.0,5.0,3.0,0.606061,0.018107
52,9.0,3.0,2.0,0.606061,0.023180
91,12.0,4.0,2.0,0.604959,0.016120
66,10.0,4.0,1.0,0.604959,0.013675
9,5.0,5.0,1.0,0.604408,0.013675
53,9.0,3.0,3.0,0.601102,0.009120
44,8.0,4.0,3.0,0.600000,0.020571
82,11.0,5.0,2.0,0.599449,0.012992
50,9.0,2.0,3.0,0.599449,0.022757


<h2>Данные с нормализацией</h2>

In [33]:
%%time
acc = []
units = 5
layers = 2
dropout = 1 #умножается на 0.1
epoch = 50

X_splitter = X_norm.copy()
y_splitter = pd.get_dummies(y)

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_splitter, y_splitter, test_size=0.2, random_state=i)
    model = Sequential()
    model.add(Dense(input_dim=X_train.shape[1], units=units,
                     kernel_initializer='normal', bias_initializer='zeros'))
    model.add(Activation('relu'))

    for i in range(0, layers):
        model.add(Dense(units=units, kernel_initializer='normal',
                         bias_initializer='zeros'))
        model.add(Activation('relu'))
        model.add(Dropout(dropout*0.1))

    model.add(Dense(units=2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epoch, verbose=0)
    acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
    print(f"accuracy: {acc[-1]}")
print(f'units: {units}, layers: {layers}, dropout: {dropout}, mean acc: {np.mean(acc)}')

12/12 [==============================] - 0s 637us/step
accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
accuracy: 0.5757575757575758
12/12 [==============================] - 0s 637us/step
accuracy: 0.5977961432506887
12/12 [==============================] - 0s 546us/step
accuracy: 0.6115702479338843
units: 5, layers: 2, dropout: 1, mean acc: 0.5895316804407713
Wall time: 9.22 s


In [34]:
%%time
X_splitter = X_norm.copy()
y_splitter = pd.get_dummies(y)
results = pd.DataFrame({'units': [], 'layers': [], 'dropout': [], 'accuracy_mean': [], 'accuracy_std': []})
for units in range(5,15):
    for layers in range(2,6):
        for dropout in range(1,4):
            acc = []
            for i in range(5):
                X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=i)
                model = Sequential()
                model.add(Dense(input_dim=X_train.shape[1], units=units,
                                 kernel_initializer='normal', bias_initializer='zeros'))
                model.add(Activation('relu'))

                for i in range(0, layers):
                    model.add(Dense(units=units, kernel_initializer='normal',
                                     bias_initializer='zeros'))
                    model.add(Activation('relu'))
                    model.add(Dropout(dropout*0.1))

                model.add(Dense(units=2))
                model.add(Activation('softmax'))

                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                model.fit(X_train, y_train, epochs=50, verbose=0)
                acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
                print(f'units: {units}, layers: {layers}, dropout: {dropout}, accuracy: {acc[-1]}')
                
            results = results.append({'units': units, 
                                      'layers': layers, 
                                      'dropout': dropout, 
                                      'accuracy_mean': np.mean(acc), 
                                      'accuracy_std': np.std(acc)}, 
                                      ignore_index=True)
            print(f'units: {units}, layers: {layers}, dropout: {dropout}, -----------------------------mean acc: {np.mean(acc)}')

12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6253443526170799
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6115702479338843
12/12 [==============================] - 0s 594us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 1, accuracy: 0.6088154269972452
units: 5, layers: 2, dropout: 1, -----------------------------mean acc: 0.6060606060606061
12/12 [==============================] - 0s 546us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 5, layers: 2, dropout: 2, accuracy: 0.628099173553719
12/12 [==============================] - 0s 637us/step
units: 5, layers: 2, dropout: 2, acc

12/12 [==============================] - 0s 637us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 637us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.5730027548209367
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 1, accuracy: 0.6060606060606061
units: 6, layers: 2, dropout: 1, -----------------------------mean acc: 0.5867768595041322
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.6225895316804407
12/12 [==============================] - 0s 546us/step
units: 6, layers: 2, dropout: 2, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 6, layers: 2, dropout: 2, ac

12/12 [==============================] - 0s 546us/step
units: 7, layers: 2, dropout: 1, accuracy: 0.6088154269972452
12/12 [==============================] - 0s 546us/step
units: 7, layers: 2, dropout: 1, accuracy: 0.5674931129476584
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 1, accuracy: 0.5950413223140496
units: 7, layers: 2, dropout: 1, -----------------------------mean acc: 0.596694214876033
12/12 [==============================] - 0s 728us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 546us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 546us/step
units: 7, layers: 2, dropout: 2, accuracy: 0.5537190082644629
12/12 [==============================] - 0s 637us/step
units: 7, layers: 2, dropout: 2, acc

12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 1, accuracy: 0.581267217630854
12/12 [==============================] - 0s 728us/step
units: 8, layers: 2, dropout: 1, accuracy: 0.6170798898071626
units: 8, layers: 2, dropout: 1, -----------------------------mean acc: 0.5994490358126722
12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 2, accuracy: 0.5647382920110193
12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 2, accuracy: 0.6198347107438017
12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 2, accuracy: 0.6005509641873278
12/12 [==============================] - 0s 637us/step
units: 8, layers: 2, dropout: 2, accuracy: 0.581267217630854
12/12 [==============================] - 0s 546us/step
units: 8, layers: 2, dropout: 2, accuracy: 0.6005509641873278
units: 8, layers: 2, dropout: 2, -----------------------------mean acc: 0.5933884297520662
1

12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 1, accuracy: 0.6088154269972452
units: 9, layers: 2, dropout: 1, -----------------------------mean acc: 0.5983471074380166
12/12 [==============================] - 0s 728us/step
units: 9, layers: 2, dropout: 2, accuracy: 0.5674931129476584
12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 2, accuracy: 0.6170798898071626
12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 2, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 2, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 2, accuracy: 0.6033057851239669
units: 9, layers: 2, dropout: 2, -----------------------------mean acc: 0.5933884297520662
12/12 [==============================] - 0s 637us/step
units: 9, layers: 2, dropout: 3, accuracy: 0.5922865013774105

12/12 [==============================] - 0s 546us/step
units: 10, layers: 2, dropout: 2, accuracy: 0.559228650137741
12/12 [==============================] - 0s 637us/step
units: 10, layers: 2, dropout: 2, accuracy: 0.6170798898071626
12/12 [==============================] - 0s 637us/step
units: 10, layers: 2, dropout: 2, accuracy: 0.5895316804407713
12/12 [==============================] - 0s 546us/step
units: 10, layers: 2, dropout: 2, accuracy: 0.5674931129476584
12/12 [==============================] - 0s 546us/step
units: 10, layers: 2, dropout: 2, accuracy: 0.5922865013774105
units: 10, layers: 2, dropout: 2, -----------------------------mean acc: 0.5851239669421486
12/12 [==============================] - 0s 637us/step
units: 10, layers: 2, dropout: 3, accuracy: 0.5757575757575758
12/12 [==============================] - 0s 546us/step
units: 10, layers: 2, dropout: 3, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 637us/step
units: 10, layers: 2, dropou

12/12 [==============================] - 0s 546us/step
units: 11, layers: 2, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 546us/step
units: 11, layers: 2, dropout: 2, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 637us/step
units: 11, layers: 2, dropout: 2, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 11, layers: 2, dropout: 2, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 637us/step
units: 11, layers: 2, dropout: 2, accuracy: 0.6143250688705234
units: 11, layers: 2, dropout: 2, -----------------------------mean acc: 0.596694214876033
12/12 [==============================] - 0s 728us/step
units: 11, layers: 2, dropout: 3, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 546us/step
units: 11, layers: 2, dropout: 3, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 637us/step
units: 11, layers: 2, dropou

12/12 [==============================] - 0s 637us/step
units: 12, layers: 2, dropout: 2, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 12, layers: 2, dropout: 2, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 546us/step
units: 12, layers: 2, dropout: 2, accuracy: 0.5895316804407713
12/12 [==============================] - 0s 637us/step
units: 12, layers: 2, dropout: 2, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 546us/step
units: 12, layers: 2, dropout: 2, accuracy: 0.6005509641873278
units: 12, layers: 2, dropout: 2, -----------------------------mean acc: 0.5933884297520662
12/12 [==============================] - 0s 637us/step
units: 12, layers: 2, dropout: 3, accuracy: 0.5757575757575758
12/12 [==============================] - 0s 637us/step
units: 12, layers: 2, dropout: 3, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 546us/step
units: 12, layers: 2, dropo

12/12 [==============================] - 0s 637us/step
units: 13, layers: 2, dropout: 2, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 546us/step
units: 13, layers: 2, dropout: 2, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 13, layers: 2, dropout: 2, accuracy: 0.6225895316804407
12/12 [==============================] - 0s 546us/step
units: 13, layers: 2, dropout: 2, accuracy: 0.5730027548209367
12/12 [==============================] - 0s 546us/step
units: 13, layers: 2, dropout: 2, accuracy: 0.5950413223140496
units: 13, layers: 2, dropout: 2, -----------------------------mean acc: 0.5928374655647384
12/12 [==============================] - 0s 637us/step
units: 13, layers: 2, dropout: 3, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 637us/step
units: 13, layers: 2, dropout: 3, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 637us/step
units: 13, layers: 2, dropo

12/12 [==============================] - 0s 637us/step
units: 14, layers: 2, dropout: 2, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 637us/step
units: 14, layers: 2, dropout: 2, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 637us/step
units: 14, layers: 2, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 637us/step
units: 14, layers: 2, dropout: 2, accuracy: 0.6115702479338843
12/12 [==============================] - 0s 546us/step
units: 14, layers: 2, dropout: 2, accuracy: 0.5840220385674931
units: 14, layers: 2, dropout: 2, -----------------------------mean acc: 0.5983471074380166
12/12 [==============================] - 0s 637us/step
units: 14, layers: 2, dropout: 3, accuracy: 0.5867768595041323
12/12 [==============================] - 0s 546us/step
units: 14, layers: 2, dropout: 3, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 637us/step
units: 14, layers: 2, dropo

In [35]:
results.sort_values('accuracy_mean', ascending=False).head(20)

,units,layers,dropout,accuracy_mean,accuracy_std
0,5.0,2.0,1.0,0.606061,0.012564
13,6.0,2.0,2.0,0.605510,0.011478
15,6.0,3.0,1.0,0.603306,0.014261
38,8.0,2.0,3.0,0.603306,0.019712
32,7.0,4.0,3.0,0.602755,0.013200
18,6.0,4.0,1.0,0.602204,0.019134
71,10.0,5.0,3.0,0.601102,0.018635
33,7.0,5.0,1.0,0.601102,0.013873
113,14.0,3.0,3.0,0.601102,0.011995
79,11.0,4.0,2.0,0.601102,0.008060


In [50]:
np.std(acc)

0.03691473126741355

Проверка не удалась(((

<h3>PCA</h3>

In [48]:
%%time
acc = []
units = 30
layers = 3
dropout = 2 #умножается на 0.1
epoch = 50

X_splitter = X_pca.copy()
y_splitter = pd.get_dummies(y)

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_splitter, y_splitter, test_size=0.2, random_state=i)
    model = Sequential()
    model.add(Dense(input_dim=X_train.shape[1], units=units,
                     kernel_initializer='normal', bias_initializer='zeros'))
    model.add(Activation('relu'))

    for i in range(0, layers):
        model.add(Dense(units=units, kernel_initializer='normal',
                         bias_initializer='zeros'))
        model.add(Activation('relu'))
        model.add(Dropout(dropout*0.1))

    model.add(Dense(units=2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epoch, verbose=0)
    acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
    print(f"accuracy: {acc[-1]}")
print(f'units: {units}, layers: {layers}, dropout: {dropout}, mean acc: {np.mean(acc)}')

12/12 [==============================] - 0s 546us/step
accuracy: 0.5564738292011019
12/12 [==============================] - 0s 546us/step
accuracy: 0.5674931129476584
12/12 [==============================] - 0s 637us/step
accuracy: 0.5757575757575758
12/12 [==============================] - 0s 637us/step
accuracy: 0.5647382920110193
12/12 [==============================] - 0s 546us/step
accuracy: 0.5702479338842975
units: 30, layers: 3, dropout: 2, mean acc: 0.5669421487603306
Wall time: 11 s


In [49]:
%%time
X_splitter = X_pca.copy()
y_splitter = pd.get_dummies(y)
results = pd.DataFrame({'units': [], 'layers': [], 'dropout': [], 'accuracy_mean': [], 'accuracy_std': []})
for units in range(2,10):
    for layers in range(1,4):
        for dropout in range(0,4):
            acc = []
            for i in range(5):
                X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=i)
                model = Sequential()
                model.add(Dense(input_dim=X_train.shape[1], units=units,
                                 kernel_initializer='normal', bias_initializer='zeros'))
                model.add(Activation('relu'))

                for i in range(0, layers):
                    model.add(Dense(units=units, kernel_initializer='normal',
                                     bias_initializer='zeros'))
                    model.add(Activation('relu'))
                    model.add(Dropout(dropout*0.1))

                model.add(Dense(units=2))
                model.add(Activation('softmax'))

                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                model.fit(X_train, y_train, epochs=50, verbose=0)
                acc.append(accuracy_score(model.predict(X_test)[:,1].round(), y_test[1]))
                print(f'units: {units}, layers: {layers}, dropout: {dropout}, accuracy: {acc[-1]}')
                
            results = results.append({'units': units, 
                                      'layers': layers, 
                                      'dropout': dropout, 
                                      'accuracy_mean': np.mean(acc), 
                                      'accuracy_std': np.std(acc)}, 
                                      ignore_index=True)
            print(f'units: {units}, layers: {layers}, dropout: {dropout}, -----------------------------mean acc: {np.mean(acc)}')

12/12 [==============================] - 0s 546us/step
units: 2, layers: 1, dropout: 0, accuracy: 0.5344352617079889
12/12 [==============================] - 0s 546us/step
units: 2, layers: 1, dropout: 0, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 546us/step
units: 2, layers: 1, dropout: 0, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 2, layers: 1, dropout: 0, accuracy: 0.6143250688705234
12/12 [==============================] - 0s 637us/step
units: 2, layers: 1, dropout: 0, accuracy: 0.5344352617079889
units: 2, layers: 1, dropout: 0, -----------------------------mean acc: 0.577961432506887
12/12 [==============================] - 0s 546us/step
units: 2, layers: 1, dropout: 1, accuracy: 0.5730027548209367
12/12 [==============================] - 0s 637us/step
units: 2, layers: 1, dropout: 1, accuracy: 0.6033057851239669
12/12 [==============================] - 0s 546us/step
units: 2, layers: 1, dropout: 1, acc

12/12 [==============================] - 0s 455us/step
units: 3, layers: 1, dropout: 0, accuracy: 0.5647382920110193
12/12 [==============================] - 0s 637us/step
units: 3, layers: 1, dropout: 0, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 3, layers: 1, dropout: 0, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 546us/step
units: 3, layers: 1, dropout: 0, accuracy: 0.5977961432506887
units: 3, layers: 1, dropout: 0, -----------------------------mean acc: 0.5867768595041323
12/12 [==============================] - 0s 546us/step
units: 3, layers: 1, dropout: 1, accuracy: 0.5895316804407713
12/12 [==============================] - 0s 637us/step
units: 3, layers: 1, dropout: 1, accuracy: 0.6253443526170799
12/12 [==============================] - 0s 546us/step
units: 3, layers: 1, dropout: 1, accuracy: 0.5895316804407713
12/12 [==============================] - 0s 637us/step
units: 3, layers: 1, dropout: 1, ac

12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 0, accuracy: 0.6060606060606061
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 0, accuracy: 0.5619834710743802
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 0, accuracy: 0.6143250688705234
units: 4, layers: 1, dropout: 0, -----------------------------mean acc: 0.590633608815427
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 1, accuracy: 0.5757575757575758
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 1, accuracy: 0.6253443526170799
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 1, accuracy: 0.6088154269972452
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 1, accuracy: 0.5619834710743802
12/12 [==============================] - 0s 546us/step
units: 4, layers: 1, dropout: 1, acc

12/12 [==============================] - 0s 546us/step
units: 5, layers: 1, dropout: 0, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 637us/step
units: 5, layers: 1, dropout: 0, accuracy: 0.6088154269972452
units: 5, layers: 1, dropout: 0, -----------------------------mean acc: 0.5961432506887052
12/12 [==============================] - 0s 637us/step
units: 5, layers: 1, dropout: 1, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 546us/step
units: 5, layers: 1, dropout: 1, accuracy: 0.628099173553719
12/12 [==============================] - 0s 546us/step
units: 5, layers: 1, dropout: 1, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 546us/step
units: 5, layers: 1, dropout: 1, accuracy: 0.559228650137741
12/12 [==============================] - 0s 546us/step
units: 5, layers: 1, dropout: 1, accuracy: 0.5840220385674931
units: 5, layers: 1, dropout: 1, -----------------------------mean acc: 0.5900826446280992
1

12/12 [==============================] - 0s 546us/step
units: 6, layers: 1, dropout: 0, accuracy: 0.6088154269972452
units: 6, layers: 1, dropout: 0, -----------------------------mean acc: 0.5922865013774106
12/12 [==============================] - 0s 637us/step
units: 6, layers: 1, dropout: 1, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 546us/step
units: 6, layers: 1, dropout: 1, accuracy: 0.6198347107438017
12/12 [==============================] - 0s 546us/step
units: 6, layers: 1, dropout: 1, accuracy: 0.5922865013774105
12/12 [==============================] - 0s 546us/step
units: 6, layers: 1, dropout: 1, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 637us/step
units: 6, layers: 1, dropout: 1, accuracy: 0.6143250688705234
units: 6, layers: 1, dropout: 1, -----------------------------mean acc: 0.6016528925619835
12/12 [==============================] - 0s 637us/step
units: 6, layers: 1, dropout: 2, accuracy: 0.5702479338842975

12/12 [==============================] - 0s 546us/step
units: 7, layers: 1, dropout: 1, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 7, layers: 1, dropout: 1, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 546us/step
units: 7, layers: 1, dropout: 1, accuracy: 0.5619834710743802
12/12 [==============================] - 0s 546us/step
units: 7, layers: 1, dropout: 1, accuracy: 0.5482093663911846
12/12 [==============================] - 0s 546us/step
units: 7, layers: 1, dropout: 1, accuracy: 0.6060606060606061
units: 7, layers: 1, dropout: 1, -----------------------------mean acc: 0.5763085399449035
12/12 [==============================] - 0s 546us/step
units: 7, layers: 1, dropout: 2, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 637us/step
units: 7, layers: 1, dropout: 2, accuracy: 0.6060606060606061
12/12 [==============================] - 0s 546us/step
units: 7, layers: 1, dropout: 2, ac

12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 1, accuracy: 0.6115702479338843
12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 1, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 1, accuracy: 0.5206611570247934
12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 1, accuracy: 0.6308539944903582
units: 8, layers: 1, dropout: 1, -----------------------------mean acc: 0.5900826446280992
12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 2, accuracy: 0.6115702479338843
12/12 [==============================] - 0s 637us/step
units: 8, layers: 1, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 546us/step
units: 8, layers: 1, dropout: 2, ac

12/12 [==============================] - 0s 637us/step
units: 9, layers: 1, dropout: 1, accuracy: 0.5950413223140496
12/12 [==============================] - 0s 637us/step
units: 9, layers: 1, dropout: 1, accuracy: 0.5730027548209367
12/12 [==============================] - 0s 637us/step
units: 9, layers: 1, dropout: 1, accuracy: 0.5895316804407713
units: 9, layers: 1, dropout: 1, -----------------------------mean acc: 0.5867768595041323
12/12 [==============================] - 0s 637us/step
units: 9, layers: 1, dropout: 2, accuracy: 0.5702479338842975
12/12 [==============================] - 0s 546us/step
units: 9, layers: 1, dropout: 2, accuracy: 0.5977961432506887
12/12 [==============================] - 0s 637us/step
units: 9, layers: 1, dropout: 2, accuracy: 0.5785123966942148
12/12 [==============================] - 0s 637us/step
units: 9, layers: 1, dropout: 2, accuracy: 0.5840220385674931
12/12 [==============================] - 0s 546us/step
units: 9, layers: 1, dropout: 2, ac

In [50]:
results.sort_values('accuracy_mean', ascending=False).head(20)

,units,layers,dropout,accuracy_mean,accuracy_std
76,8.0,2.0,0.0,0.605510,0.010655
77,8.0,2.0,1.0,0.604959,0.017301
67,7.0,2.0,3.0,0.604408,0.024242
57,6.0,3.0,1.0,0.603306,0.016063
92,9.0,3.0,0.0,0.603306,0.009383
91,9.0,2.0,3.0,0.602755,0.012369
49,6.0,1.0,1.0,0.601653,0.013451
88,9.0,2.0,0.0,0.601653,0.020138
60,7.0,1.0,0.0,0.601102,0.023978
32,4.0,3.0,0.0,0.600551,0.025725
